In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('Classificacao com Spark')\
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [4]:
spark

In [5]:
dataset = spark.read.parquet('Dataset Modelo/', inferSchema = True)

In [7]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Conjuge: integer (nullable = true)
 |-- Dependentes: integer (nullable = true)
 |-- TelefoneFixo: integer (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = true)
 |-- SegurancaOnline: integer (nullable = true)
 |-- BackupOnline: integer (nullable = true)
 |-- SeguroDispositivo: integer (nullable = true)
 |-- SuporteTecnico: integer (nullable = true)
 |-- TVaCabo: integer (nullable = true)
 |-- StreamingFilmes: integer (nullable = true)
 |-- ContaCorreio: integer (nullable = true)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContrato_DoisAnos

In [8]:
dataset.toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,9465,0,60,103.614223,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
2,2122,0,12,75.850000,0,0,0,1,1,0,...,1,0,0,0,1,0,0,0,0,1
3,3997,0,69,61.450000,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
4,6654,0,7,86.500000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,1476,0,2,20.250000,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,1
10344,1138,0,11,65.150000,0,1,1,1,1,1,...,1,0,0,1,0,0,1,0,0,0
10345,5125,0,11,94.000000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
10346,2376,0,11,61.250000,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


# Regressão Logistica

##### A regressão logística é um método popular para prever uma resposta categórica, podemos prever um resultado binário usando regressão logiística binominal ou podemos prever um resultado multiclasse usando regressão logistica multinominal